In [27]:
!pip install requests


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
!pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import os
import requests
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Access the API key
news_api_key = os.getenv("NEWS_API_KEY")

def fetch_news(topic="technology", language="en", page_size=5):
    """
    Fetches the latest news articles from NewsAPI based on topic and language, limited to a specific number of articles.
    
    Parameters:
        topic (str): Category of news to fetch (default is "technology").
        language (str): Language code for the articles (default is "en" for English).
        page_size (int): Number of articles to fetch (default is 5).
        
    Returns:
        list: A list of dictionaries, each containing details of a news article.
    """
    url = "https://newsapi.org/v2/top-headlines"
    params = {
        "apiKey": news_api_key,
        "category": topic,
        "language": language,
        "pageSize": page_size  # Limit number of articles
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        data = response.json()
        articles = data.get("articles", [])
        
        formatted_articles = [
            {
                "title": article["title"],
                "description": article["description"],
                "url": article["url"],
                "publishedAt": article["publishedAt"]
            }
            for article in articles
        ]
        return formatted_articles
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching news: {e}")
        return []
    except ValueError as e:
        print("Error processing JSON response:", e)
        return []

In [30]:
# Load Hugging Face API key from environment variable
huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")

def summarize_text_with_huggingface(text):
    """
    Summarizes the given text using Hugging Face's Inference API for a summarization model.
    
    Parameters:
        text (str): The text to be summarized.
        
    Returns:
        str: A summarized version of the input text.
    """
    # Define the Hugging Face model endpoint for summarization
    api_url = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
    headers = {"Authorization": f"Bearer {huggingface_api_key}"}
    payload = {"inputs": text}
    
    try:
        # Make the API request
        response = requests.post(api_url, headers=headers, json=payload)
        response.raise_for_status()
        
        # Extract the summary from the response
        summary = response.json()[0]["summary_text"]
        return summary
    except requests.exceptions.RequestException as e:
        print(f"Error during summarization: {e}")
        return ""

In [31]:
def fetch_and_summarize_news(topic="technology", language="en"):
    """
    Fetches the latest news articles on a given topic and provides summaries.
    
    Parameters:
        topic (str): Category of news to fetch (default is "technology").
        language (str): Language code for the articles (default is "en" for English).
        
    Returns:
        list: A list of dictionaries, each containing the title, summary, and link to a news article.
    """
    # Fetch the latest news articles
    articles = fetch_news(topic=topic, language=language)
    
    summarized_articles = []
    
    # Summarize each article's content
    for article in articles:
        title = article['title']
        description = article['description']
        url = article['url']
        
        # Combine title and description for a more comprehensive summary
        if description:
            text_to_summarize = f"{title}. {description}"
        else:
            text_to_summarize = title  # Use title alone if description is missing
        
        # Get summary
        summary = summarize_text_with_huggingface(text_to_summarize)
        
        # Store summarized article
        summarized_articles.append({
            "title": title,
            "summary": summary,
            "url": url
        })
    
    return summarized_articles

# Test the complete function
news_summaries = fetch_and_summarize_news()

Title: PSA: don't stand still in Black Ops 6's Zombie mode - Eurogamer
Summary: Call of Duty Black Ops 6 players are calling on Activision to suspend the over zealous AFK timer in its Zombie mode. PSA: don't stand still in Black ops 6's Zombie mode - Eurogamer. Call of Duty 6 is out now on PS4, Xbox One and PC.
Read more: https://www.eurogamer.net/psa-dont-stand-still-in-black-ops-6s-zombie-mode

--------------------------------------------------------------------------------

Title: Apple’s new M4 Max is the fastest CPU on the market, and we haven’t even seen M4 Ultra - 9to5Mac
Summary: Apple's new M4 Max is the fastest CPU on the market, and we haven't even seen M4 Ultra. Apple unveiled its new MacBook Pro models this week, with M4, M4 Pro, and M4Max chipsets. The new M 4...
Read more: https://9to5mac.com/2024/11/02/apple-m4-max-geekbench-macbook-pro/

--------------------------------------------------------------------------------

Title: Dragon Age: The Veilguard DLC plans unknown 

In [33]:
from IPython.display import display, Markdown

def display_summaries(news_summaries):
    """
    Displays a list of summarized articles in a structured format using Markdown.
    
    Parameters:
        summarized_articles (list): List of dictionaries containing title, summary, and URL for each article.
    """
    for article in news_summaries:
        title = article["title"]
        summary = article["summary"]
        url = article["url"]
        
        # Display article information in Markdown format
        display(Markdown(f"**Title:** [{title}]({url})"))
        display(Markdown(f"**Summary:** {summary}"))
        display(Markdown("---"))  # Divider line between articles

# Run the display function on the summarized articles
display_summaries(news_summaries)

**Title:** [PSA: don't stand still in Black Ops 6's Zombie mode - Eurogamer](https://www.eurogamer.net/psa-dont-stand-still-in-black-ops-6s-zombie-mode)

**Summary:** Call of Duty Black Ops 6 players are calling on Activision to suspend the over zealous AFK timer in its Zombie mode. PSA: don't stand still in Black ops 6's Zombie mode - Eurogamer. Call of Duty 6 is out now on PS4, Xbox One and PC.

---

**Title:** [Apple’s new M4 Max is the fastest CPU on the market, and we haven’t even seen M4 Ultra - 9to5Mac](https://9to5mac.com/2024/11/02/apple-m4-max-geekbench-macbook-pro/)

**Summary:** Apple's new M4 Max is the fastest CPU on the market, and we haven't even seen M4 Ultra. Apple unveiled its new MacBook Pro models this week, with M4, M4 Pro, and M4Max chipsets. The new M 4...

...

**Summary:** Someone has Doom running on Alarmo now - Eurogamer. Earlier this week, Tom told us Nintendo Alarma couldn't yet run Doom, but did confirm it could display a picture of a c… of a Doom game. Of course someone has Doomrunning on Alarmso now.

---

**Title:** [Kindle Colorsoft owners complain of a yellow bar on the e-reader’s screen - The Verge](https://www.theverge.com/2024/11/2/24286289/kindle-colorsoft-yellow-bar-display-e-reader)

**Summary:**  Kindle Colorsoft owners complain of a yellow bar on the e-reader’s screen - The Verge. People who’ve ordered the Kindle Color soft Signature Edition say there's a discolored yellow bar at the bottom of the display. They say the yellow bar appears on the screen when the device is turned on.

---